In [1]:
from src.model_utils import ImplicitTransBridge

In [5]:
from transformers import AutoTokenizer
llm_path = "/home/nfs04/model/Qwen2.5/Qwen2.5-0.5B-Instruct"
llm_path_a = "/home/nfs04/model/Qwen2.5/Qwen2.5-0.5B-Instruct"
llm_path_b = "/home/nfs04/model/Qwen2.5/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_path)
tokenizer.add_special_tokens({"bos_token": "<bos>"})
model = ImplicitTransBridge(llm_path, max_gen_len=10, llm_bos_token_id=tokenizer.bos_token_id,
                            llm_pad_token_id=tokenizer.pad_token_id, stage=0,
                            slm_path_a=llm_path_a, slm_path_b=llm_path_b)
model.cuda().eval()

10/16/2024 17:45:00 - INFO - src.model_utils.modeling_itb - Small LM A model size: 494.032768 M
10/16/2024 17:45:00 - INFO - src.model_utils.modeling_itb - mapping a layer size: 3.214848 M


ImplicitTransBridge(
  (llm): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151936, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm()
          (post_attention_layernorm): Qwen2RMSNorm()
      

In [3]:
xx  = "我喜欢你"
ids_xx = tokenizer([xx], return_tensors="pt")
prompt = "Translate from Chinese to English: "
ids_prompt = tokenizer([prompt], return_tensors="pt")
en = "I like you"
ids_en = tokenizer([en], return_tensors="pt")

In [6]:
res  = model(input_ids_prompt=ids_prompt['input_ids'], attention_mask_prompt=ids_prompt['attention_mask'],
                      input_ids_xx=ids_xx['input_ids'], attention_mask_xx=ids_xx['attention_mask'],
                      input_ids_en=ids_en['input_ids'], attention_mask_en=ids_en['attention_mask'])

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [28]:
res.logits[0][13]

tensor([13.9015,  9.8175,  5.6632,  ..., -1.6009, -1.6007, -1.6008],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [31]:
tokenizer(['sss'], add_special_tokens=True)

{'input_ids': [[84160]], 'attention_mask': [[1]]}

In [33]:
tokenizer.pad({"input_ids": [[12,3]]}, )

TypeError: PreTrainedTokenizerBase.pad() got an unexpected keyword argument 'add_special_tokens'

In [1]:
import json
with open("./stage0_isen.json", "r") as f:
    data = json.load(f)

In [3]:
with open("./is_en.en.txt", "w") as f:
    for d in data:
        f.write(d['hyp'] + "\n")

In [21]:
ids_en['input_ids']

tensor([[    40,   1075,    498, 151643, 151643, 151643, 151643, 151643],
        [    45,  52091,    287,    374,    279,   6722,    315,   5616]])

In [10]:
xx  = ["我喜欢你", "南京是中国的首都"]
ids_xx = tokenizer(xx, return_tensors="pt", padding=True, truncation=True)
prompt = "Translate from Chinese to English: "
ids_prompt = tokenizer([prompt] * 2, return_tensors="pt", padding=True, truncation=True)
en = ["I like you", "Nanjing is the capital of China"]
ids_en = tokenizer(en, return_tensors="pt", padding=True, truncation=True)
res_a  = model(input_ids_prompt=ids_prompt['input_ids'], attention_mask_prompt=ids_prompt['attention_mask'],
                      input_ids_xx=ids_xx['input_ids'], attention_mask_xx=ids_xx['attention_mask'],
                      input_ids_en=ids_en['input_ids'], attention_mask_en=ids_en['attention_mask'])

In [1]:
with open("/home/nfs02/wangzj/dataset/opus-100-corpus/v1.0/supervised/en-is/opus.en-is-train.is", "r") as f:
    xx = []
    for k in f.readlines():
        xx.append(k.strip())
with open("/home/nfs02/wangzj/dataset/opus-100-corpus/v1.0/supervised/en-is/opus.en-is-train.en", "r") as f:
    en = []
    for k in f.readlines():
        en.append(k.strip())

In [2]:
with open("./data/opus_en_is.jsonl", "w") as f:
    import json
    for a, b in zip(xx, en):
        f.write(json.dumps({"src": b, "tgt": a}, ensure_ascii=False) + "\n")